In [1]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
yf.pdr_override() # <== thats all it takes :-)
!pip install monthly-returns-heatmap
import monthly_returns_heatmap as mrh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for monthly-returns-heatmap: filename=monthly_returns_heatmap-0.0.11-py3-none-any.whl size=8112 sha256=2c9fad22844c7ea2a12be3ff3b998a3a72dd3334e4c99e71a056ccd292ce2a4c
  Stored in directory: /root/.cache/pip/wheels/a8/68/93/e7887114a1f27ed095dbb50aed4d59bd66341e43a990725711
Successfully built monthly-returns-heatmap


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
df=pd.read_csv("/content/gdrive/MyDrive/NIFTYDATA/2023_June_Fifty_index.csv")

In [4]:
df=df[df['datetime'].str.contains('2023')].reset_index(drop=True)

In [5]:
#!pip install ta

In [6]:
#import ta

In [7]:
# df['TSI']=ta.momentum.TSIIndicator(df['close'])

In [8]:
# TRUE STRENGTH INDEX CALCULATION

def get_tsi(close, long, short, signal):
    diff = close - close.shift(1)
    abs_diff = abs(diff)

    diff_smoothed = diff.ewm(span = long, adjust = False).mean()
    diff_double_smoothed = diff_smoothed.ewm(span = short, adjust = False).mean()
    abs_diff_smoothed = abs_diff.ewm(span = long, adjust = False).mean()
    abs_diff_double_smoothed = abs_diff_smoothed.ewm(span = short, adjust = False).mean()

    tsi = (diff_double_smoothed / abs_diff_double_smoothed) * 100
    signal = tsi.ewm(span = signal, adjust = False).mean()

    return tsi, signal


In [9]:
df.head(1)

,datetime,symbol,open,high,low,close,volume
0,2023-01-02 09:15:00,NSE:NIFTY,18133.65,18153.2,18091.35,18101.85,31542815.0


In [10]:
df['tsi'], df['signal_line'] = get_tsi(df['close'], 25, 13, 13)

In [11]:
df['Date']= df['datetime'].str.split(" ",expand=True)[0]

In [12]:
all_dates = yf.download('^NSEI',
                        start='2023-03-01',
                        end='2023-06-25',
                        progress=False)
all_dates=all_dates.reset_index()
all_dates['Date']=all_dates['Date'].astype(str)
all_dates_list=list(all_dates['Date'])

# skip two months - as we need to skip minimum 25 as per TSI setting

In [13]:
def determine_signals(data):
    position = 0  # Position in the market: 0 for out, 1 for long, -1 for short
    signals = []

    for i in range(0, len(data)):
        if data['tsi'][i] > data['signal_line'][i] and position !=1 :
          signals.append('Buy')
          position = 1

        elif data['tsi'][i] < data['signal_line'][i] and position != -1:
          signals.append('Sell')
          position = -1

        else:
          signals.append('Hold')

    return signals

In [20]:
final = pd.DataFrame(columns=['datetime','signals','close','PNL'])

for day in tq.tqdm(all_dates_list):
  data1=df[df['Date'] == day].reset_index().drop('index',axis=1) # selecting day from list


  trading_signals = determine_signals(data1) #generating signals
  data1[ "signals" ]= trading_signals

  data2 = data1['signals'] # steps to decide exit at 3pm
  data2=data2[data2.str.contains('Buy|Sell')].reset_index(drop=True)
  last=data2[len(data2)-1]

  if last == "Buy":
    data1['signals'][23] = "Sell"
  else:
    data1['signals'][23] = "Buy"

  data1=data1[['datetime','signals','close']] # take required trades
  data1=data1[data1['signals'].str.contains('Buy|Sell')].reset_index(drop=True)


  PNL = [] # PNL Calculation
  for i in range(0,len(data1)-1):
    if data1['signals'][i] == "Buy":
      money = ((data1['close'][i+1] - data1['close'][i]) * 50) - 50 # multiply point to lot and deduct tax
    else:
      money = ((data1['close'][i] - data1['close'][i+1]) * 50) - 50
    PNL.append(money)

  data1=data1[:len(PNL)] # take only trades as the last position is exit
  data1['PNL'] = PNL

  #limiting max trades per day
  per_day_trades = 3
  if len(data1) > per_day_trades:
    data1=data1[:per_day_trades]

  # martingle if 1st trade goes wrong
  if len(data1) > 1:
    if data1['PNL'][0] < 0:
      data1['PNL'][1] = (data1['PNL'][1] * 2) - 50  # deduct 2nd lot tax

    # if second trade also goes wrong then continue with double quantity else single quantity
    if len(data1) > 2:
      if data1['PNL'][1] < 0:
        data1['PNL'][2] = (data1['PNL'][2] * 2) - 50

  final=pd.concat([final,data1]).reset_index(drop=True) #concating new enteries into existing dataframe

# seperate hours
final['hours']=final['datetime'].str.split(" ",expand=True)[1].str.split(":",expand=True)[0]
final['hours'] = final['hours'].astype(int)

# stop trading after x time
timing = 15
print('PNL: ',sum(final[final['hours'] < timing]['PNL']))

  0%|          | 0/77 [00:00<?, ?it/s]

PNL:  70847.50000000057


In [21]:
print("Total percentage since march: ", int(round((sum(final[final['hours'] < timing]['PNL'])/300000) * 100,0)),"%")

Total percentage since march:  24 %


In [23]:
final=final.drop('hours',axis=1)

In [24]:
final.to_csv("TSI_test.csv",index=False)

In [25]:
from google.colab import files
files.download('TSI_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>